# Cheerio boat mass and bubble height estimation

### Intro:
The goal of this document is to adapt the heigth of the air pocket on the cheerioboat such that the exit port of the boat is at the surface level. To do this we will estimate the mass of the cheerioboat.

We will make a few assumptions:
- The boat is printed using solid infill density.
- The density of the used filament stays constant.
- Any mass lost by holes used for propulsion is negligable.
- The boat is circularly symmetric.
- The floatation of the boat is in equilibrium.
- Surface tension forces do not affect the floatation of the boat.


### Parameters:
<img src="cheerio_dimensions_annotated.png" width="300"><br>
|Parameter|Definition|
|------|------|
| $R$:  | The radius of the cylindrical part of the boat. | 
| $\text{d}$:  | The thickness of the walls and floors. | 
| $s$:  | The height of the air pocket. | 
| $c$:  | The height of the ring where the fuel channel resides in. | 
| $\text{l}$:  | The lenght of the of the channel ring. | 
| $h$:  | The height of the fuel containing cone. | 
| $n$:  | The height of the neck of the cone. | 
| $o$:  | The radius of the opening of the cone.  | 
| $\rho$:| The density of the filament. |

### Calculations:
To make calculations more compact we introduce some more values: <br>
The slope and slope angle of the cheerio boat cone: $\beta = \frac{d+o-R+\text{l}}{h-n-c}$, $\theta = \arctan{-\beta}$.<br>
A linear equation relating the radius of the cone to its height from the top of the air pocket: $r(z) = \beta z+R-l-\beta c$.<br>

We will sum the volumes of the individual components from bottom to top:<br>
$\begin{align}
    V &= \pi \text{d} R^2 &&\text{bottom slab of air bubble}\\% bottom slab air bubble
      &+ \pi s \left(2\text{d}R-\text{d}^2\right) &&\text{air bubble wall}\\ % air bubble wall
      &+ \pi \text{d} R^2 &&\text{top slab of air bubble}\\% top slab air bubble &
      &+ \pi c \left(2\text{l}R-\text{l}^2\right) &&\text{channel ring}\\ 
      &+ \int_{0}^{h-n}\int_{0}^{2\pi}\int_{r(z)-\frac{d}{\cos{(\theta)}}}^{r(z)}rdrd\phi dz &&\text{cone}\\
      &- \int_{0}^{c}\int_{0}^{2\pi}\int_{R-\text{l}}^{r(z)}rdrd\phi dz  &&\text{ring \& cone overlap correction}\\
      &+ \pi n \left(2\text{d}o+\text{d}^2\right) &&\text{neck ring}
\end{align}$

Evaluating the integrals and filling in $\theta$, we get:
$\begin{align}
    V &= \pi \text{d} R^2 &&\text{bottom slab of air bubble}\\% bottom slab air bubble
      &+ \pi s \left(2\text{d}R-\text{d}^2\right) &&\text{air bubble wall}\\ % air bubble wall
      &+ \pi \text{d} R^2 &&\text{top slab of air bubble}\\% top slab air bubble &
      &+ \pi c \left(2\text{l}R-\text{l}^2\right) &&\text{channel ring}\\ 
      &+ \pi\text{d}\left(h-n\right)\sqrt{1+\beta^2}\left(2R-\beta\left(2c-h+n\right)-\text{d}\sqrt{1+\beta^2}-2\text{l}\right) &&\text{cone}\\
      &- \frac{\pi}{3}\beta c^2\left(\beta c +3 \text{l} - 3R\right)  &&\text{ring \& cone overlap correction}\\
      &+ \pi n \left(2\text{d}o+\text{d}^2\right) &&\text{neck ring}
\end{align}$

using $\cos\left(\arctan\left(-\beta\right)\right) = \frac{1}{\sqrt{1+\beta^2}}$.

In [1]:
from sympy.parsing.latex import parse_latex
from sympy.solvers import solve
from sympy import Symbol
from numpy import pi

# waardes in mm
values = {
    'R' : 12,
    'pi': pi,
    'l' : 2,
    'c' : 1.2,
    'n' : 1,
    'o' : 2.5,
    'd' : 1,
    'h' : 12,
    's' : 5.5, 
    'rho' : 0.00126, # g/mm^3 
    'mu' : 0.000997, # g/mm^3
    'extra_mass' : 1 # g
}

values['b'] = (values['d'] + values['o'] - values['R'] + values['l']) / (values['h'] - values['n'] - values['c']) # beta

# oude formule
volume_eq = r"""
  \pi d*R^2
+ \pi s*(2d*R-d^2)
+ \pi d*R^2
+ \pi c*(2l*R-l^2)
+ \pi d*(h-n)\sqrt{1+b^2}(2R-b*(2c-h+n)-d*\sqrt{1+b^2}-2l)
- \frac{\pi}{3}b c^2(b*c +3 l - 3R)
+ \pi n*(2d*o+d^2)
"""

parts = {
    'bubble_slab_bot': r'\pi*d*R^2',
    'bubble_wall': r'\pi*s*(2d*R-d^2)',
    'bubble_slab_top': r'\pi*d*R^2',
    'channel_ring': r'\pi*c*(2l*R-l^2)',
    'cone': r'(\int_{0}^{h-n}\int_{0}^{2\pi}\int_{b*z+R-l-b*c-d*\sqrt{1+b^2}}^{b*z+R-l-b*c}r drd\phi dz)',
    'cone_corr': r'-(\int_{0}^{c}\int_{0}^{2\pi}\int_{R-l}^{b*z+R-l-b*c}r drd\phi dz)',
    'neck' : r'\pi*n*(2d*o+d^2)'
}

volume_expression  = parse_latex('+'.join(parts.values()))

volume_values = {symb:values[str(symb)] for symb in volume_expression.free_symbols} # 
s_symbol = [x for x in volume_expression.free_symbols if str(x)=='s'][0]

volume_values.pop(s_symbol) # s weer een vrije variabele maken

evaluated_expression = volume_expression.subs(volume_values).doit()
print(evaluated_expression)

72.2566310325652*s + 1601.76424006201


#### Archimedes' principle:
Now that we have the mass of the cheerio boat as a function of the height of the air bubble. We can use archimedes principle to find the optimal height such that the exit port lies at the water surface.
$$m_{boat}g - \mu_{water} g V_{boat} = 0$$
$$m_{boat} = \mu_{water} V_{boat}$$
With $\mu_{water}$ the denity of the water. $V_{boat}$ is the volume of water that the boat displaces. We will assume that this wil always be a cylinder. If we take this volume to be the volume of a cylinder that reaches to the bottom of the outlet we get:
$$V_{boat} = \pi R^2 (d + s + d)$$

In [2]:
displacement_expression = parse_latex(r"""\mu\pi*R^2*(d + s + d)""")

displacement_values = {symb:values[str(symb)] for symb in displacement_expression.free_symbols}
displacement_values.pop(s_symbol) # s weer een vrije variabele maken

archimedes_expression = values['rho'] * evaluated_expression - displacement_expression.subs(displacement_values) + values['extra_mass']

print(archimedes_expression)
print(solve(archimedes_expression, s_symbol))


2.11615859429697 - 0.359988818989547*s
[5.87840089099661]
